# Exercise 02. Join

In [16]:
import pandas as pd
import sqlite3

## Create a connection to the database using the `sqlite3` library

In [17]:
con = sqlite3.connect("../data/checking-logs.sqlite")

## Create a new table called `datamart` in the database by joining the tables `pageviews` and `checker` using only one query

In [18]:
con.execute("DROP TABLE IF EXISTS datamart")

query = """
CREATE TABLE IF NOT EXISTS datamart AS
    SELECT ch.uid AS uid, labname, timestamp AS first_commit_ts,
        datetime AS first_view_ts
    FROM checker ch 
    LEFT JOIN pageviews pv
    ON pv.uid = ch.uid
    WHERE ch.uid LIKE 'user_%'
    AND status = 'ready'
    AND numTrials = 1
    AND labname IN 
    (
        'laba04', 'laba04s', 'laba05',
        'laba06', 'laba06s', 'project1'
    )
    AND 
    (
        pv.datetime = 
        (
            SELECT MIN(datetime)
            FROM pageviews
            WHERE pageviews.uid=ch.uid
        )
        OR pv.datetime IS NULL
    )
"""
con.execute(query)
datamart = pd.io.sql.read_sql('SELECT * FROM datamart', con, 
                       parse_dates=['first_commit_ts', 'first_view_ts'])
datamart


,uid,labname,first_commit_ts,first_view_ts
0,user_4,project1,2020-04-17 05:19:02.744528,NaT
1,user_4,laba04,2020-04-17 11:33:17.366400,NaT
2,user_4,laba04s,2020-04-17 11:48:41.992466,NaT
3,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
4,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
...,...,...,...,...
135,user_23,laba06,2020-05-21 08:34:10.517205,NaT
136,user_19,laba06s,2020-05-21 13:27:06.705881,2020-04-21 20:30:38.034966
137,user_23,laba06s,2020-05-21 14:29:15.709568,NaT
138,user_17,laba06,2020-05-21 15:21:31.567615,2020-04-18 10:56:55.833899


## Using Pandas methods, create two dataframes: `test` and `control`

In [19]:
con.execute("DROP TABLE IF EXISTS test")
con.execute("DROP TABLE IF EXISTS control")

test = datamart[datamart['first_view_ts'].notnull()]
control = datamart[datamart['first_view_ts'].isnull()]
control = control.fillna(test['first_view_ts'].mean())

test.info()
control.info()

test.to_sql('test', con)
control.to_sql('control', con)

<class 'pandas.core.frame.DataFrame'>
Index: 59 entries, 3 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              59 non-null     object        
 1   labname          59 non-null     object        
 2   first_commit_ts  59 non-null     datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 2.3+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 81 entries, 0 to 137
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              81 non-null     object        
 1   labname          81 non-null     object        
 2   first_commit_ts  81 non-null     datetime64[ns]
 3   first_view_ts    81 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 3.2+ KB


81

## Close the connection

In [20]:
con.close()